<a href="https://colab.research.google.com/github/anxosanchez/SOPQ-2324/blob/main/Minimizaci%C3%B3n-da-Enerx%C3%ADa-Libre-de-Gibbs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neste **notebook** analizamos a minimización directa da **enerxía libre de Gibbs** das especies, sen supostos sobre a estequiometría das reaccións. Só aplicamos a restrición de conservación dos átomos. Usamos o **NIST Webbook** para proporcionar os datos da enerxía de Gibbs de cada especie.

Como recordatorio, consideramos o equilibrio entre as especies $CO$, $H_2O$, $CO_2$ e $H_2$, a 1000K, e unha presión total de 5 atm cun caudal equimolar inicial de $CO$ e $H_2O$.

Imos almacenar todos os datos e cálculos en vectores, polo que debemos asignar cada posición do vector a unha especie. Aquí están as definicións que utilizamos neste traballo.

1.  CO  
2.  H<sub>2</sub>O  
3.  CO<sub>2</sub>  
4.  H<sub>2</sub>

In [1]:
# @title
import numpy as np

T = 842  # K
R = 8.314e-3 # kJ/mol/K

P = 5.37 # atm, this is the total pressure in the reactor
Po = 1.0 # atm, this is the standard state pressure

In [2]:
# @title
species = ['CO', 'H2O', 'CO2', 'H2']

Agora, constrúuiremos a función de **enerxía libre de Gibbs**, explicando o cambio de actividade debido aos cambios de concentración (mestura ideal).

Impoñemos a restrición de que todos os átomos se conserven desde as condicións iniciais ata a distribución en equilibrio das especies. Estas restricións teñen a forma de $A_{eq} n = b_{eq}$, onde $n$ é o vector dos números moles de cada especie.

In [3]:
# @title
# Heats of formation at 298.15 K

Hf298 = [
    -110.53,  # CO
    -241.826, # H2O
    -393.51,  # CO2
       0.0]   # H2


Agora estamos preparados para resolver o problema.

## 1 Cálculo das fraccións molares e presións parciais

As presións aquí están de acordo coas presións atopadas por outros métodos. O pequeno desacordo (no terceiro ou cuarto decimal) débese probablemente a tolerancias de converxencia nos diferentes algoritmos utilizados.

In [4]:
# @title
# Shomate parameters for each species
#           A          B           C          D          E            F          G       H
WB = [[25.56759,  6.096130,     4.054656,  -2.671301,  0.131021, -118.0089, 227.3665,   -110.5271],  # CO
      [30.09200,  6.832514,     6.793435,  -2.534480,  0.082139, -250.8810, 223.3967,   -241.8264],  # H2O
      [24.99735,  55.18696,   -33.69137,    7.948387, -0.136638, -403.6075, 228.2431,   -393.5224],  # CO2
      [33.066178, -11.363417,  11.432816,  -2.772874, -0.158558, -9.980797, 172.707974,    0.0]]     # H2

WB = np.array(WB)

In [5]:
# @title
# Shomate equations

t = T/1000

T_H = np.array([t,  t**2 / 2.0, t**3 / 3.0, t**4 / 4.0, -1.0 / t, 1.0, 0.0, -1.0])
T_S = np.array([np.log(t), t,  t**2 / 2.0,  t**3 / 3.0, -1.0 / (2.0 * t**2), 0.0, 1.0, 0.0])

H = np.dot(WB, T_H)        # (H - H_298.15) kJ/mol
S = np.dot(WB, T_S/1000.0) # absolute entropy kJ/mol/K

Gjo = Hf298 + H - T*S      # Gibbs energy of each component at 1000 K


In [6]:
# @title
def func(nj):
    nj = np.array(nj)
    Enj = np.sum(nj);
    Gj =  Gjo / (R * T) + np.log(nj / Enj * P / Po)
    return np.dot(nj, Gj)

In [7]:
# @title
Aeq = np.array([[ 1,    0,    1,    0],  # C balance
                [ 1,    1,    2,    0],  # O balance
                [ 0,    2,    0,    2]]) # H balance

In [8]:
# @title
# equimolar feed of 1 mol H2O and 1 mol CO
beq = np.array([1,  # mol C fed
                2,  # mol O fed
                2]) # mol H fed

In [9]:
# @title
def ec1(nj):
    'restriccións da lei de conservaxción de átomos'
    return np.dot(Aeq, nj) - beq

In [10]:
# @title
from scipy.optimize import fmin_slsqp

n0 = [0.5, 0.5, 0.5, 0.5]  # initial guesses
N = fmin_slsqp(func, n0, f_eqcons=ec1)
print (N)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -99.94067284451523
            Iterations: 3
            Function evaluations: 16
            Gradient evaluations: 3
[0.35799314 0.35799314 0.64200686 0.64200686]


## 1 Cálculo das fraccións molares e presións parciais

As presións aquí están de acordo cas presións atopadas por outros métodos. O pequeno desacordo (no terceiro ou cuarto decimal) débese probablemente a tolerancias de converxencia nos diferentes algoritmos utilizados.

In [11]:
# @title
yj = N / np.sum(N)
Pj = yj * P

for s, y, p in zip(species, yj, Pj):
    print (s, y, p)

CO 0.17899656952306864 0.9612115783388786
H2O 0.17899656952306864 0.9612115783388786
CO2 0.32100343047693136 1.7237884216611215
H2 0.32100343047693136 1.7237884216611215


## 2. Cálculo de constantes de equilibrio
Podemos calcular a constante de equilibrio para a reacción:
  
$$
CO+H_2O⇌CO_2+H_2
$$
    
ten en en conta que para definir unha constante de equilibrio é necesario especificar unha reacción, aínda que nin sequera sexa necesario considerar unha reacción para obter a distribución en equilibrio das especies.

In [12]:
# @title
nuj = np.array([-1, -1, 1, 1])  # stoichiometric coefficients of the reaction
K = np.prod(yj**nuj)
print('A constante de equilibrio (adimensional) vale: {:5.4f}'.format(K))

A constante de equilibrio (adimensional) vale: 3.2161
